In [6]:
# Imports

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import pyomo
import pyomo.opt
import pyomo.environ as pe

from src.parsers import HMParser

In [2]:
# Data parsing

data = HMParser(file_path='data/EC_V4.xlsx', ec_id=1)
data.parse()

In [40]:
# Auxiliary function to convert numpy arrays to dictionaries

def convert_to_dictionary(a):
    temp_dictionary = {}

    if len(a.shape) == 3:
        for dim0 in np.arange(a.shape[0]):
            for dim1 in np.arange(a.shape[1]):
                for dim2 in np.arange(a.shape[2]):
                    temp_dictionary[(dim0+1, dim1+1, dim2+1)] = a[dim0, dim1, dim2]
    elif len(a.shape) == 2:
        for dim0 in np.arange(a.shape[0]):
            for dim1 in np.arange(a.shape[1]):
                temp_dictionary[(dim0+1, dim1+1)] = a[dim0, dim1]

    else:
        for dim0 in np.arange(a.shape[0]):
            temp_dictionary[(dim0+1)] = a[dim0]

    return temp_dictionary

In [73]:
# Create model and attribute sets

model = pe.ConcreteModel()

# Set default behaviour
default_behaviour = pe.Constraint.Skip

# Variable iterators
model.t = pe.Set(initialize=np.arange(1, data.generator['p_forecast'].shape[1] + 1),
                 doc='Time periods')
model.stor = pe.Set(initialize=np.arange(1, data.storage['p_charge_limit'].shape[0] + 1),
                    doc='Number of storage units')
model.v2g = pe.Set(initialize=np.arange(1, data.vehicle['p_charge_max'].shape[0] + 1),
                   doc='Number of V2G units')
model.cs = pe.Set(initialize=np.arange(1, data.charging_station['p_charge_limit'].shape[0] + 1),
                  doc='Number of charging stations')

In [74]:
# Create generator properties

# Sets and Parameters
model.gen = pe.Set(initialize=np.arange(1, data.generator['p_forecast'].shape[0] + 1),
                   doc='Number of generators')
model.genType = pe.Param(model.gen,
                         initialize=convert_to_dictionary(data.generator['type_generator'].astype(int)),
                         doc='Types of generators')
model.genMin = pe.Param(model.gen,
                        initialize=convert_to_dictionary(data.generator['p_min']),
                        doc='Minimum power generation')
model.genMax = pe.Param(model.gen, model.t,
                        initialize=convert_to_dictionary(data.generator['p_forecast']),
                        doc='Forecasted power generation')

# Variables
model.genActPower = pe.Var(model.gen, model.t, within=pe.NonNegativeReals, initialize=0,
                           doc='Active power generation')
model.genExcPower = pe.Var(model.gen, model.t, within=pe.NonNegativeReals, initialize=0,
                           doc='Excess power generation')
model.genXo = pe.Var(model.gen, model.t, within=pe.Binary, initialize=0,
                     doc='Generation on/off')

# Constraints
# Maximum generation
def _genActMaxEq(m, g, t):
    if m.genType[g] == 1:
        return m.genActPower[g, t] <= m.genMax[g, t]
    elif m.genType[g] == 2:
        return m.genActPower[g, t] + m.genExcPower[g, t] == m.genMax[g, t]
    return default_behaviour
model.genActMaxEq = pe.Constraint(model.gen, model.t, rule=_genActMaxEq,
                                  doc='Maximum active power generation')
# Minimum generation
def _genActMinEq(m, g, t):
    if m.genType[g] == 1:
        return m.genActPower[g, t] >= m.genMin[g] * m.genXo[g, t]
    return default_behaviour
model.genActMinEq = pe.Constraint(model.gen, model.t, rule=_genActMinEq,
                                  doc='Minimum active power generation')

In [75]:
# Create load properties

# Sets and Parameters
model.loads = pe.Set(initialize=np.arange(1, data.load['p_forecast'].shape[0] + 1),
                    doc='Number of loads')
model.loadMax = pe.Param(model.loads, model.t,
                         initialize=convert_to_dictionary(data.load['p_forecast'] * 5),
                         doc='Forecasted power consumption')
model.loadRedMax = pe.Param(model.loads, model.t,
                            initialize=convert_to_dictionary(data.load['p_forecast'] * 0.5),
                            doc='Maximum power reduction')
model.loadCutMax = pe.Param(model.loads, model.t,
                            initialize=convert_to_dictionary(data.load['p_forecast'] * 0.5),
                            doc='Maximum power cut')

# Variables
model.loadRedActPower = pe.Var(model.loads, model.t, within=pe.NonNegativeReals, initialize=0,
                               doc='Active power reduction')
model.loadCutActPower = pe.Var(model.loads, model.t, within=pe.NonNegativeReals, initialize=0,
                               doc='Active power cut')
model.loadENS = pe.Var(model.loads, model.t, within=pe.NonNegativeReals, initialize=0,
                       doc='Energy Not Supplied')
model.loadXo = pe.Var(model.loads, model.t, within=pe.Binary, initialize=0,
                      doc='Load on/off')

# Constraints
def _loadRedActEq(m, l, t):
    return m.loadRedActPower[l, t] <= m.loadRedMax[l, t]
model.loadReactEq = pe.Constraint(model.loads, model.t, rule=_loadRedActEq,
                                  doc='Maximum active power reduction')

def _loadCutActEq(m, l, t):
    return m.loadCutActPower[l, t] == m.loadCutMax[l, t] * m.loadXo[l, t]
model.loadCutActEq = pe.Constraint(model.loads, model.t, rule=_loadCutActEq,
                                   doc='Maximum active power cut')

def _loadENSEq(m, l, t):
    return m.loadENS[l, t] + m.loadRedActPower[l, t] + m.loadCutActPower[l, t] <= m.loadMax[l, t]
model.loadENSEq = pe.Constraint(model.loads, model.t, rule=_loadENSEq,
                                doc='Maximum energy not supplied')